In [169]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm.notebook import tqdm, trange

In [4]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
referer = 'https://www.resilience.org/latest-articles/'
headers = {'User-Agent': user_agent, 'referer':referer}

In [123]:
topic_page = 'https://grist.org/all-topics/'
response = requests.get(topic_page, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
topics = ['https://grist.org' + e['href'] for e in soup.find_all('a', {'class':'topic-listing__topic-link'})]

In [126]:
# Remove some topics that don't work
topics.pop(2)
topics.pop(13)
topics.pop(14)
topics.pop(14)
topics.pop(27)
topics.pop(31)
topics.pop(31)

'https://grist.org/video/'

In [ ]:
article_links = []
for topic in tqdm(topics):
    
    response = requests.get(topic, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    last_page = int(soup.find_all('li', {'class':'pagination__page'})[-1].text \
                    .replace("\n", "").replace(",", ""))

    for page in trange(1, last_page+1):
        url = topic + 'page/{}/'.format(page)

        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        new_links = [e['href'] for e in soup.find_all('a', {'class':'tease__link'})]
        article_links.extend(new_links)

In [172]:
article_links_unique = list(set(article_links))
print(len(article_links))
print(len(article_links_unique))

64182
58345


In [184]:
def get_article_data(article):

    art_dict = {'url':article}
    
    response = requests.get(article, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        art_dict['title']    = soup.find('h1', {'class':'topper-headings__title'}).text \
                                   .replace("\n", "")
    except:
        pass
        #print("Missing title")
    
    try:
        art_dict['subtitle'] = soup.find('h2', {'class':'topper-headings__subtitle'}).text \
                                   .replace("\n", "")
    except:
        pass
        #print("Missing subtitle")
    
    try:
        art_dict['author']   = soup.find('span', {'class':'contributor-info__name'}).text \
                                   .replace("\n", "")
    except:
        pass
        #print("Missing author")
    
    try:
        art_dict['date']     = soup.find('dd', {'class':'article-meta__item-value'}).text
    except:
        pass
        #print("Missing date")
    
    try:
        art_dict['text']     = " ".join([p.text for p in soup.find('div', {'class':'article-body js-hang-punc'}) \
                                                            .find_all('p')])
        #if art_dict['text'] == '':
        #    print("Missing text")
    except:
        pass
        #print("Missing text")    
    
    return art_dict

In [ ]:
grist = []

for article in tqdm(article_links_unique):
    grist.append(get_article_data(article))

gr = pd.DataFrame(grist)

  0%|          | 0/58345 [00:00<?, ?it/s]

In [180]:
len(grist)

32481

In [181]:
gr.head()

,url,title,subtitle,author,date,text
0,https://grist.org/living/this-guy-built-an-aut...,This guy built an authentic passenger ra...,To make this subterranean transit-geek wonderl...,Sarah Laskow,"Mar 18, 2013","Kingston SubJason Shron, a self-described “tra..."
1,https://grist.org/article/2010-03-08-cocaine-a...,Cocaine addicts are snorting their way t...,"Passetti via Flickr You may snort at the news,...",Ashley Braun,"Mar 09, 2010","Passetti via FlickrYou may snort at the news, ..."
2,https://grist.org/article/2010-04-04-undergrou...,Underground school lunch blogger hits ‘G...,"Still life, with spork: Mrs. Q’s snap of pre-f...",Tom Philpott,"Apr 05, 2010","Still life, with spork: Mrs. Q’s snap of pre-f..."
3,https://grist.org/article/becker-homeland/,The Homeland Is (Cough) Secure (Cough),NaN,Suzy Becker,"Feb 17, 2004",
4,https://grist.org/article/be7/,"Pretty Soon, They'll Be Selling Decaf Wa...",Rivers and groundwater are being contaminated ...,Grist staff,"Mar 29, 2000",Rivers and groundwater are being contaminated...


In [182]:
gr.to_pickle("../Data/Grist/grist.pkl")